In [1]:
import pandas as pd
import backtrader as bt
import backtrader as bt
import time

In [4]:
# 创建Cerebro引擎  
cerebro = bt.Cerebro() 
#Cerebro引擎在后台创建broker(经纪人)，系统默认资金量为10000 
# 引擎运行前打印期出资金  
print('组合期初资金: %.2f' % cerebro.broker.getvalue()) 
cerebro.run() 
# 引擎运行后打期末资金  
print('组合期末资金: %.2f' % cerebro.broker.getvalue())


组合期初资金: 10000.00
组合期末资金: 10000.00


In [3]:
import requests 
import backtrader as bt
import backtrader.analyzers as btanalyzers
import json 
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import ccxt 

In [5]:

exchange = ccxt.okex({
    'apiKey': '1e9e4338-c701-4345-b00f',
    'secret': '3C59D08A6572CC1DDB4FC28',
    'password': '123456',
    'timeout': 30000,
    'enableRateLimit': True,
})

In [6]:
ticker = exchange.fetch_ticker(symbol="BTC/USDT")
ticker

RequestTimeout: okex GET https://www.okx.com/api/v5/asset/currencies

In [7]:
a =1

In [11]:
def get_binance_bars(symbol, interval, startTime, endTime):
 
    url = "https://api.binance.com/api/v3/klines"
 
    startTime = str(int(startTime.timestamp() * 1000))
    endTime = str(int(endTime.timestamp() * 1000))
    limit = '1000'
 
    req_params = {"symbol" : symbol, 'interval' : interval, 'startTime' : startTime, 'endTime' : endTime, 'limit' : limit}
 
    df = pd.DataFrame(json.loads(requests.get(url, params = req_params).text))
 
    if (len(df.index) == 0):
        return None
     
    df = df.iloc[:, 0:6]
    df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
 
    df.open      = df.open.astype("float")
    df.high      = df.high.astype("float")
    df.low       = df.low.astype("float")
    df.close     = df.close.astype("float")
    df.volume    = df.volume.astype("float")
    
    df['adj_close'] = df['close']
     
    df.index = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.datetime]
 
    return df

In [13]:
df_list = []
# 数据起点时间
last_datetime = dt.datetime(2020, 11, 23)
while True:
    new_df = get_binance_bars('ETHUSDT', '1m', last_datetime, dt.datetime.now()) # 获取1分钟k线数据
    
    if new_df is None:
        break
    df_list.append(new_df)
    last_datetime = max(new_df.index) + dt.timedelta(0, 1)
 
df = pd.concat(df_list)
df.shape


ConnectionError: HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v3/klines?symbol=ETHUSDT&interval=1m&startTime=1606060800000&endTime=1681315959986&limit=1000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f77966d1760>: Failed to establish a new connection: [Errno 101] Network is unreachable'))